In [11]:
import nltk
import re
import string
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
from nltk.tokenize import word_tokenize as nltk_word_tokenize
from nltk.corpus import stopwords
import numpy as np
from heapq import nlargest

from string import punctuation
from collections import Counter
import os

import pickle
import pandas as pd

from process_text import sent_tokenize, clean_text, cleanup_sentence, clean_number, cleanup_sentences
from generate_keywords import get_training_keywords, get_section_keywords
from generate_summary import get_summary_training_keywords, get_summary_section_keywords

In [12]:
VALIDATION_DATASET = False
TEST_DATASET = True

In [13]:
## Best Parameters
GEN_SUMMARY_TRAIN = True
NUM_NEIGHBOURS_TRAINING_KEYWORD_SUMMARY = 0
NUM_NEIGHBOURS_SECTION_KEYWORD_SUMMARY = 20
NUM_SENTS_TRAINING_KEYWORD_SUMMARY = 90
NUM_SENTS_SECTION_KEYWORD_SUMMARY = 20

In [14]:
train_dir = '../../../Dataset/FNS2022/Greek/training/'
if VALIDATION_DATASET:
    dir_ = '../../../Dataset/FNS2022/Greek/validation/'

if TEST_DATASET:
    dir_ = '../../../Dataset/FNS2022/Greek/testing/'
    

annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
system_summary_dir = 'greek_summaries'
lang = 'el'

multi_lingual_cleaned_toc_file = 'multi_lingual_cleaned_toc_sections.pkl'

team_name1 = 'SSC_AI_RG'
team_name = 'summary'


In [15]:
dir_

'../../../Dataset/FNS2022/Greek/testing/'

In [6]:
#if GEN_SUMMARY_TRAIN:
 #   training_keywords = get_training_keywords(train_dir, gold_summary_dir)
  #  pickle.dump(training_keywords, open('training_keywords.pkl', 'wb'))

In [16]:
if GEN_SUMMARY_TRAIN:
    training_keywords = pickle.load(open('training_keywords.pkl', 'rb'))

In [17]:
section_keywords =  get_section_keywords(multi_lingual_cleaned_toc_file, 'clean_greek')
section_keywords

{'δήλωση': 684.9274879331334,
 'προέδρου': 612.6424081550396,
 'διευθύνοντος': 553.8154921032855,
 'συμβούλου': 553.8154921032855,
 'καλύτερες': 448.4229749294871,
 'στιγμές': 448.4229749294871,
 'επιθεώρηση': 368.7408026508831,
 'μια': 171.90833296083144,
 'ματιά': 171.90833296083144,
 'αρχή': 93.9458197743788,
 'έκθεση': 58.31777855208011,
 'μας': 46.82504364830046,
 'στρατηγική': 39.42653784446979,
 'επιστολή': 20.18658177247684,
 'πληροφορίες': 14.293872493871422,
 'ανακοίνωση': 14.283659169984391,
 'συνέλευσης': 14.283659169984391,
 'εισαγωγή': 12.928826013437424,
 'αξιολόγηση': 12.419915633304502,
 'αναθεώρηση': 12.09417345424946,
 'ετήσιας': 11.418823752850953,
 'γενικής': 11.418823752850953,
 'οικονομικός': 11.002595349761808,
 'απολογισμός': 11.002595349761808,
 'επιχειρηματική': 10.565792343992566,
 'κριτική': 10.565792343992566,
 'επιχειρηματικό': 10.055501846341159,
 'μοντέλο': 10.055501846341159,
 'σφαιρικη': 9.763240090167402,
 'εικονα': 9.763240090167402,
 'κανω': 8.5999

In [18]:
from rouge_score import rouge_scorer

In [19]:
num_file = 0

os.makedirs(system_summary_dir)
for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        #print("Processing File Number: ", num_file)
        num_file = num_file +1 
        file_id = file.split('.')[0]
   
        with open(os.path.join(dir_,annual_reports_dir, str(file_id)+'.txt'), "r", encoding='utf-8') as report_file:
            tmp_file = []
            for line in report_file:
                line = line.replace("\n","").replace("\t"," ").replace("\xa0"," ")
                tmp_file.append(line)
            text = ' '.join(tmp_file)
            if GEN_SUMMARY_TRAIN:
                raw_sentences, clean_sentences = cleanup_sentences(text, lang)
                summary = get_summary_training_keywords(training_keywords, clean_sentences, raw_sentences, NUM_SENTS_TRAINING_KEYWORD_SUMMARY, NUM_NEIGHBOURS_TRAINING_KEYWORD_SUMMARY)
                raw_sentences, clean_sentences = cleanup_sentences(summary, lang)
            else:
                raw_sentences, clean_sentences = cleanup_sentences(text, lang)
            #summary = get_summary(clean_sentences, raw_sentences, NUM_SENTS_SECTION_KEYWORD_SUMMARY, NUM_NEIGHBOURS)
            summary = get_summary_section_keywords(section_keywords, clean_sentences, raw_sentences, NUM_SENTS_SECTION_KEYWORD_SUMMARY, NUM_NEIGHBOURS_SECTION_KEYWORD_SUMMARY)
            summary_split = summary.split(' ')
            if len(summary_split) > 1000:
                summary = ' '.join(summary_split[:1000])
                summary_split =  summary.split(' ')
            #print('Length of Summary: ', len(summary_split))  
            
        with open(os.path.join(system_summary_dir, file_id+'_'+team_name+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))
            
        if ".DS_Store" in file:
            continue
    except Exception as e:
        print(file, e)

In [ ]:
gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'])

rouge_scores = {}
rouge_scores['rouge-1'] = {}
rouge_scores['rouge-2'] = {}

rouge_scores['rouge-1']['p'] = 0
rouge_scores['rouge-1']['r'] = 0
rouge_scores['rouge-1']['f'] = 0
rouge_scores['rouge-2']['p'] = 0
rouge_scores['rouge-2']['r'] = 0
rouge_scores['rouge-2']['f'] = 0
num_file = 0

for file in os.listdir(system_summary_dir):
    try:
        if num_file%10 == 0:
            print("Processing File Number: ", num_file)
        file_id = file.split('_')[0]
        if ".DS_Store" in file:
            continue
            
        with open(os.path.join(system_summary_dir,file), "r",  encoding='utf-8') as f:
            system_summary_txt = f.read()
            #print(file, 'summary len: ', len(system_summary_txt.split(' ')))
                
        sum_rouge_scores = {}
        sum_rouge_scores['rouge-1'] = {}
        sum_rouge_scores['rouge-2'] = {}
        sum_rouge_scores['rouge-L'] = {}

        sum_rouge_scores['rouge-1']['p'] = 0
        sum_rouge_scores['rouge-1']['r'] = 0
        sum_rouge_scores['rouge-1']['f'] = 0
        sum_rouge_scores['rouge-2']['p'] = 0
        sum_rouge_scores['rouge-2']['r'] = 0
        sum_rouge_scores['rouge-2']['f'] = 0
        
        num_sum_file = 0
        
        for sum_file in os.listdir(gold_summary_dir_):
            if ".DS_Store" in file:
                continue
            if os.path.isfile(os.path.join(gold_summary_dir_,sum_file)) and file_id==sum_file.split("_")[0]:
                #sum_file.split("_")[1].split('.')[0] == str(1)
                
                with open(os.path.join(gold_summary_dir_,sum_file), "r",  encoding='utf-8') as f:
                    gold_sum_txt = f.read()
                    sum_scores = scorer.score(gold_sum_txt, system_summary_txt)
                    sum_rouge_scores['rouge-1']['p'] = sum_rouge_scores['rouge-1']['p'] + sum_scores['rouge1'][0]
                    sum_rouge_scores['rouge-1']['r'] = sum_rouge_scores['rouge-1']['r'] + sum_scores['rouge1'][1]
                    sum_rouge_scores['rouge-1']['f'] = sum_rouge_scores['rouge-1']['f'] + sum_scores['rouge1'][2]
                    sum_rouge_scores['rouge-2']['p'] = sum_rouge_scores['rouge-2']['p'] + sum_scores['rouge2'][0]
                    sum_rouge_scores['rouge-2']['r'] = sum_rouge_scores['rouge-2']['r'] + sum_scores['rouge2'][1]
                    sum_rouge_scores['rouge-2']['f'] = sum_rouge_scores['rouge-2']['f'] + sum_scores['rouge2'][2]
                    num_sum_file = num_sum_file + 1
            
        sum_rouge_scores['rouge-1']['p'] = sum_rouge_scores['rouge-1']['p'] / (num_sum_file)
        sum_rouge_scores['rouge-1']['r'] = sum_rouge_scores['rouge-1']['r'] / (num_sum_file)
        sum_rouge_scores['rouge-1']['f'] = sum_rouge_scores['rouge-1']['f'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['p'] = sum_rouge_scores['rouge-2']['p'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['r'] = sum_rouge_scores['rouge-2']['r'] / (num_sum_file)
        sum_rouge_scores['rouge-2']['f'] = sum_rouge_scores['rouge-2']['f'] / (num_sum_file)
        
        
        rouge_scores['rouge-1']['p'] = rouge_scores['rouge-1']['p'] + sum_rouge_scores['rouge-1']['p']
        rouge_scores['rouge-1']['r'] = rouge_scores['rouge-1']['r'] + sum_rouge_scores['rouge-1']['r']
        rouge_scores['rouge-1']['f'] = rouge_scores['rouge-1']['f'] + sum_rouge_scores['rouge-1']['f']
        rouge_scores['rouge-2']['p'] = rouge_scores['rouge-2']['p'] + sum_rouge_scores['rouge-2']['p']
        rouge_scores['rouge-2']['r'] = rouge_scores['rouge-2']['r'] + sum_rouge_scores['rouge-2']['r']
        rouge_scores['rouge-2']['f'] = rouge_scores['rouge-2']['f'] + sum_rouge_scores['rouge-2']['f']
        
        num_file = num_file + 1
        
        
    except Exception as e:
        print(e)
        pass

In [ ]:
num_file

In [ ]:
rouge_scores['rouge-1']['p'] = rouge_scores['rouge-1']['p'] / (num_file)
rouge_scores['rouge-1']['r'] = rouge_scores['rouge-1']['r'] / (num_file)
rouge_scores['rouge-1']['f'] = rouge_scores['rouge-1']['f'] / (num_file)
rouge_scores['rouge-2']['p'] = rouge_scores['rouge-2']['p'] / (num_file)
rouge_scores['rouge-2']['r'] = rouge_scores['rouge-2']['r'] / (num_file)
rouge_scores['rouge-2']['f'] = rouge_scores['rouge-2']['f'] / (num_file)
rouge_scores


<b>Best Hyper Parameters</b>

Using Training Summary Keywords = True
Neighbours in training keyword summary = 0
Neighbours in section keyword summary = 20
Sents to be extracted using training keywords = 90
Sents to be extracted using section keywords = 20

{'rouge-1': {'p': 0.3545679162864239,
  'r': 0.3562884244233777,
  'f': 0.13941338686986218},
 'rouge-2': {'p': 0.14677692898396094,
  'r': 0.13604481107105668,
  'f': 0.0553798033750471}}


Sentences in section keyword summary

Sents = 30 - No Change

Sents = 20 - No Change

Sents = 10 - No Change

Sents = 5 - No Change

Sents = 3 - No Change

Sents = 2

{'rouge-1': {'p': 0.3545679162864239, 'r': 0.3562884244233777, 'f': 0.13941338686986218}, 'rouge-2': {'p': 0.14677692898396094, 'r': 0.13604481107105668, 'f': 0.0553798033750471}}

Sents = 1

{'rouge-1': {'p': 0.3561472266312515,
  'r': 0.35511577354484447,
  'f': 0.13890647314128798},
 'rouge-2': {'p': 0.14970963629480663,
  'r': 0.13588236792615738,
  'f': 0.055415714866075085}}




-----

Sentences in training keyword summary

Sents = 200

{'rouge-1': {'p': 0.3596772846672904,
  'r': 0.34747721922127417,
  'f': 0.1361564959553662},
 'rouge-2': {'p': 0.15478568542341342,
  'r': 0.13397870114542676,
  'f': 0.05447080730515626}}


Sents = 150

{'rouge-1': {'p': 0.35202349762762786,
  'r': 0.35465894780702534,
  'f': 0.139439762956279},
 'rouge-2': {'p': 0.15087035854358902,
  'r': 0.13487819084785488,
  'f': 0.05528128981020495}}



Sents = 125

{'rouge-1': {'p': 0.35138754782769543,
  'r': 0.3555242394295787,
  'f': 0.139507258440134},
 'rouge-2': {'p': 0.14439520176483367,
  'r': 0.13188315143869847,
  'f': 0.05390663700623632}}


Sents = 100


Sents = 90

{'rouge-1': {'p': 0.3545679162864239,
  'r': 0.3562884244233777,
  'f': 0.13941338686986218},
 'rouge-2': {'p': 0.14677692898396094,
  'r': 0.13604481107105668,
  'f': 0.0553798033750471}}


Sents = 80


Sents = 75


Sents = 70


Sents = 60


Sents = 50


Sents = 20


----


Neighbours in training keyword summary


Neighbours = 0

{'rouge-1': {'p': 0.3545679162864239, 'r': 0.3562884244233777, 'f': 0.13941338686986218}, 'rouge-2': {'p': 0.14677692898396094, 'r': 0.13604481107105668, 'f': 0.0553798033750471}}

Neighbours = 3

{'rouge-1': {'p': 0.39029611380759927,
  'r': 0.33867073082641747,
  'f': 0.13537199263995583},
 'rouge-2': {'p': 0.17880338877735635,
  'r': 0.12428298695931518,
  'f': 0.05068183721135767}}


Neighbours = 5


Neighbours = 10


Neighbours = 20


Neighbours = 30

{'rouge-1': {'p': 0.4205419239400293,
  'r': 0.29960260870127775,
  'f': 0.132010906749608},
 'rouge-2': {'p': 0.19177750043127595,
  'r': 0.11189540415508503,
  'f': 0.0460422347526639}}


------


Neighbours in section keyword summary

Neighbours = 0

{'rouge-1': {'p': 0.3630691437660769,
  'r': 0.35506545298037095,
  'f': 0.12950581692016352},
 'rouge-2': {'p': 0.14979958880599736,
  'r': 0.14136921905519592,
  'f': 0.0477663564214739}}



Neighbours = 1

{'rouge-1': {'p': 0.3640565168550636,
  'r': 0.33270483170365106,
  'f': 0.1289135868784309},
 'rouge-2': {'p': 0.14050919059121456,
  'r': 0.12179989472499347,
  'f': 0.04489353219974865}}


Neighbours = 2

{'rouge-1': {'p': 0.36658061501712247,
  'r': 0.3390055489959274,
  'f': 0.13268696654109047},
 'rouge-2': {'p': 0.13678337925039202,
  'r': 0.11373918404622865,
  'f': 0.043712280538860095}}


Neighbours = 10

{'rouge-1': {'p': 0.3526176691613147,
  'r': 0.34417086352152637,
  'f': 0.1351069865852246},
 'rouge-2': {'p': 0.13476711722169407,
  'r': 0.12532161144723664,
  'f': 0.048044269458083946}}


Neighbours = 15

{'rouge-1': {'p': 0.35273986999198775,
  'r': 0.35370239577781704,
  'f': 0.13679424934912907},
 'rouge-2': {'p': 0.14411521336330885,
  'r': 0.13552066545122712,
  'f': 0.05125558569145748}}


Neighbours = 20

{'rouge-1': {'p': 0.3545679162864239,
  'r': 0.3562884244233777,
  'f': 0.13941338686986218},
 'rouge-2': {'p': 0.14677692898396094,
  'r': 0.13604481107105668,
  'f': 0.0553798033750471}}


Neighbours = 25

{'rouge-1': {'p': 0.3520788887014154,
  'r': 0.36015446512863064,
  'f': 0.13831388756926516},
 'rouge-2': {'p': 0.14768501761029046,
  'r': 0.14206092045573593,
  'f': 0.05517734994014211}}


Neighbours = 30

{'rouge-1': {'p': 0.3369455139281395,
  'r': 0.3599956739662419,
  'f': 0.13613002257329965},
 'rouge-2': {'p': 0.13179986858212878,
  'r': 0.1346887381379393,
  'f': 0.050671861490388215}}


----------------------

With and Without using Trainig Summary Keywords

Without using Training Summary Keywords

{'rouge-1': {'p': 0.4286407832838535,
  'r': 0.30593646204532676,
  'f': 0.129853451655569},
 'rouge-2': {'p': 0.19238382210008584,
  'r': 0.11090580434016743,
  'f': 0.04436444123318398}}

Using Training Summary Keywords

{'rouge-1': {'p': 0.3731743033001233,
  'r': 0.3367648145305215,
  'f': 0.13218422850631678},
 'rouge-2': {'p': 0.14528492126483272,
  'r': 0.11664752545335143,
  'f': 0.04669879204381441}}